In [13]:
%load_ext autoreload
%autoreload 2

import csv
from py.Dataset import Dataset
from py.Session import Session, MotionImage, LapseImage
from datetime import datetime

DIR = '/home/AMMOD_data/camera_traps/BayerWald/Vielkadaver-Projekt/'

ds = Dataset(DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found 32 sessions


In [11]:
def parse_datetime(datetime_str: str) -> datetime:
    return datetime.strptime(datetime_str[:-5], "%Y-%m-%dT%H:%M:%S")

assert parse_datetime("2021-05-28T09:55:24+0100") == datetime(2021, 5, 28, 9, 55, 24)

In [17]:
with open("Kadaverbilder_leer.csv", newline="") as csvfile:
    reader = csv.reader(csvfile, delimiter=";")
    header = next(reader)
    print("Header: ", header)
    session_col = header.index("deploymentID")
    datetime_col = header.index("timestamp")
    session_name: str = None
    session: Session = None
    successful = 0
    skipped = 0
    less_found = 0
    ambiguous = 0
    results = []
    last_date = None
    last_date_occurences = 0
    for row in reader:
        # print(row)
        # open session if not already opened
        if session_name != row[session_col]:
            session_name = row[session_col]
            print(f"Trying to create session {session_name}...")
            try:
                session = ds.create_session(session_name)
            except ValueError as e:
                print(e)
                session = None
        if session is None:
            skipped += 1
        else:
            # get datetime
            date = parse_datetime(row[datetime_col])
            if date == last_date:
                last_date_occurences += 1
            else:
                # look for images
                images = session.get_motion_images_from_date(last_date)

                if len(images) < last_date_occurences:
                    less_images += last_date_occurences
                elif len(images) > last_date_occurences:
                    ambiguous += last_date_occurences
                else:
                    for image in images:
                        results.append([session.name, image.filename])
                    successful += last_date_occurences

                last_date = date
                last_date_occurences = 1
print(f"Successfully found {successful} images")
print(f"Skipped {skipped} images because session was not found")
print(f"For {not_found} session dates less images were found than specified (inconsistency)")
print(f"For {ambiguous} session dates more images were found than specified (ambiguous)")


Header:  ['observationID', 'deploymentID', 'sequenceID', 'mediaID', 'timestamp', 'observationType', 'cameraSetup', 'taxonID', 'scientificName', 'count', 'countNew', 'lifeStage', 'sex', 'behaviour', 'individualID', 'classificationMethod', 'classifiedBy', 'classificationTimestamp', 'classificationConfidence', 'comments', '_id']
Trying to create session VIELAAS_Spring_Session03-VIELAAS_Beaver_03...
There are no sessions matching this name: []
Trying to create session VIELAAS_Spring_Session03-VIELAAS_Ermine_03...
There are no sessions matching this name: []
Trying to create session VIELAAS_Spring_Session03-VIELAAS_Fox_03...
Session 'Fox_03' at folder: /home/AMMOD_data/camera_traps/BayerWald/Vielkadaver-Projekt/VIELAAS_Spring_Session03-VIELAAS_Fox_03
Loaded scans.
Generating motion map...
Trying to create session VIELAAS_Spring_Session03-VIELAAS_Rat_03...
There are no sessions matching this name: []
Successfully found 3695 images
Skipped 138 images because session was not found
For 0 sessio

In [20]:
import numpy as np
np.save("test_images.npy", results)